# Import Important Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import splitfolders
from tensorflow import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from keras.layers import *
from keras.preprocessing import *
from keras.preprocessing.image import *
from tensorflow import keras
import pandas as pd
import numpy as np
import seaborn as sns
import random as random
import tensorflow_addons as tfa
import matplotlib as mpl
import warnings
import os
import cv2

# Ignore warnings

In [ ]:
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Set Random Values

In [ ]:
seed_value=40
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Count Number Of Images

In [ ]:
ImagePath = './Kideny'

In [ ]:
from pathlib import Path

Data_Root = Path(ImagePath)

Total = 0
for sub_dir in Data_Root.iterdir():
    Count = len(list(sub_dir.iterdir()))
    Total += Count
    print(f'{sub_dir.name}: {Count}')

print('')
print(f'Total Numbers Of Files: {Total}')

# Split The Data Into Train, Test, And Validation

In [ ]:
directory = "Data" 
parent_dir = '/Users/osamaanmar/Downloads/Kideny Disease Classification/'
path = os.path.join(parent_dir, directory)
os.mkdir(path)

In [ ]:
Input = ImagePath
Output = './Data'

splitfolders.ratio(Input, output=Output, seed=seed_value, ratio=(.8, 0.1, .1))

# Generate DataFrame And CSV Files

In [ ]:
directory = '''CSV''' 
parent_dir = '/Users/osamaanmar/Downloads/Kideny Disease Classification/'

path = os.path.join(parent_dir, directory)
os.mkdir(path)

In [ ]:
def dataframe_csv_creator(input_path,output_path ,name):
    in_path = input_path
    out_path = output_path
    file_name = name
    
    Image_Path = []
    Labels = []

    folders = os.listdir(in_path)
    for fold in folders:
        folderpath = os.path.join(in_path, fold)
        filelist = os.listdir(folderpath)
        for file in filelist:
            fpath = os.path.join(folderpath, file)
            Image_Path.append(fpath)
            Labels.append(fold)

    Data = pd.DataFrame({'ImagesPath':Image_Path, 'Label':Labels})

    Data.to_csv(os.path.join(out_path,r'{}.csv'.format(file_name)) ,index=False)

    return Data

In [ ]:
Train_DF = dataframe_csv_creator('./Data/train/', './CSV', 'Train')

In [ ]:
Test_DF = dataframe_csv_creator('./Data/test/', './CSV', 'Test')

In [ ]:
Val_DF = dataframe_csv_creator('./Data/val/', './CSV', 'Val')

# Pie Chart For Image Label

In [ ]:
Train_DF['Label'].value_counts().plot.pie(autopct = '%0.1f%%',
                                    textprops={'size':'x-large'},
                                    shadow = True,
                                    title = "Images Label Pie Chart",
                                    figsize = (7,7),
                                    ylabel = 'Count');

# Count Plot For Image Label

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.countplot(x=Train_DF["Label"]).set(title='Images Label Count Plot')
plt.show()

# Data Visualization 

In [ ]:
def plot_some_images(Data, NumberOfImages): 

    RandomSamples=Data.sample(NumberOfImages)
    NumberOfImages = NumberOfImages
    
    Row  = int(tf.get_static_value(math.ceil(NumberOfImages**0.5)))
    Col  = int(tf.get_static_value(math.ceil(NumberOfImages**0.5)))

    plt.figure(figsize= (20, 20))
    for i,(index,row) in enumerate(RandomSamples.iterrows()):
        plt.subplot(Row,Col,i+1)
        img=plt.imread(row['ImagesPath'])
        plt.imshow(img)
        plt.title(row['Label'])
        plt.axis('off')

**1. Training Data**

In [ ]:
plot_some_images(Train_DF, 20)

**2. Testing Data**

In [ ]:
plot_some_images(Test_DF, 20)

**3. Validation Data**

In [ ]:
plot_some_images(Val_DF, 20)

# Generate Image Data

In [ ]:
Training_Data = ImageDataGenerator(1. /255,
                                   vertical_flip=True,
                                  horizontal_flip=True,
                                 )

Testing_Data = ImageDataGenerator(1. /255)

Validation_Data = ImageDataGenerator(1. /255)

In [ ]:
Training_Set = Training_Data.flow_from_dataframe(dataframe=Train_DF,
                                               x_col='ImagesPath',
                                               y_col='Label',
                                               class_mode='categorical',
                                               seed=seed_value,
                                               target_size=(160, 160),
                                               batch_size=32,
                                               shuffle=True)

In [ ]:
Testing_Set = Testing_Data.flow_from_dataframe(dataframe=Test_DF,
                                               x_col='ImagesPath',
                                               y_col='Label',
                                               class_mode='categorical',
                                               seed=seed_value,
                                               target_size=(160, 160),
                                               batch_size=32,
                                               shuffle=True)

In [ ]:
Validation_Set = Validation_Data.flow_from_dataframe(dataframe=Val_DF,
                                               x_col='ImagesPath',
                                               y_col='Label',
                                               class_mode='categorical',
                                               seed=seed_value,
                                               target_size=(160, 160),
                                               batch_size=32,
                                               shuffle=True)

# Models

In [ ]:
Input_Shape = (160, 160, 3)
Epochs = 100
Number_Of_Classes = len(os.listdir(ImagePath))

In [ ]:
Callback = tf.keras.callbacks.EarlyStopping(
                    monitor="accuracy",
                    patience=5,
                    verbose=0,
                    mode="auto",
                    restore_best_weights=True,
                    start_from_epoch=10,
                )

In [ ]:
class Models(): 
    def __init__(self,input_shape,epochs) :
        self.input_shape = input_shape
        self.epochs = epochs
        
    def convolutional_neural_network(self):

        model = keras.models.Sequential([
            
                                         Conv2D(32, 3, activation='relu', input_shape = self.input_shape),
                                         keras.layers.MaxPooling2D(),
                                         
                                         keras.layers.Conv2D(32, 3, activation='relu'),
                                         keras.layers.MaxPooling2D(),

                                         keras.layers.Conv2D(64, 3, activation='relu'),
                                         keras.layers.MaxPooling2D(),
                                        
                                         keras.layers.Flatten(),
                                         Dropout(0.2),

                                         keras.layers.Dense(128 ,activation='relu'),
                                        
                                         keras.layers.Dense(Number_Of_Classes, activation='softmax'),

                                         ])
            
        model.compile(loss= 'categorical_crossentropy',
                        optimizer='ADAM',
                        metrics=['accuracy'])
        
        model.fit(Training_Set, 
                  epochs= Epochs, 
                  validation_data=Validation_Set,
                  callbacks=[Callback],
                  shuffle=True
                    );
        
        return model
    
    
    def tranfer_learning(self, base_model):  

        base_model = base_model
        base_model.trainable = False
            
        inputs = tf.keras.Input(self.input_shape)
        x = base_model(inputs)
        x = GlobalAveragePooling2D()(x)
        x = Flatten()(x)
        x = Dropout(0.2)(x)
        outputs = Dense(Number_Of_Classes, 'sigmoid')(x)
        model = tf.keras.Model(inputs, outputs)
            
            
        model.compile(loss= 'categorical_crossentropy',
                        optimizer='Adam',
                        metrics=['accuracy'])

        model.fit(Training_Set,
                  epochs=self.epochs,
                  validation_data=Testing_Set,
                  callbacks=[Callback], 
                  shuffle=True,
                  );
        
        return model    

# Model Result Visualization 

In [ ]:
def model_result_visualization(model):

    history = model.history.history
    fig, ax = plt.subplots(1, 2, figsize=(20, 10))
   

    for i, metrics in enumerate(['loss', 'accuracy']):
        ax[i].plot(history[metrics])
        ax[i].plot(history['val_' + metrics])
        ax[i].set_title('Model {}'.format(metrics))
        ax[i].set_xlabel('epochs')
        ax[i].set_ylabel(metrics)
        ax[i].legend(['train', 'val'])

# Model Evaluation

In [ ]:
def model_evulation(model, data):
    loss, accuracy = model.evaluate(data, verbose=0)
    
    return loss, accuracy

# Generate Real And Prediction Classes

In [ ]:
def generate_y(model):
    Real = Testing_Set.classes
    Prediction = model.predict(Testing_Set)
    Prediction = np.argmax(Prediction, axis=1)
    
    return Real, Prediction


# Confusion Matrix

In [ ]:
def confusion_matrix(Real, Prediction):
        ClassesName = Testing_Set.class_indices
        ConfusionMatrix = tf.math.confusion_matrix(Real,Prediction).numpy()
        ConfusionMatrix = pd.DataFrame(ConfusionMatrix,
                     index = ClassesName, 
                     columns = ClassesName)
        return ConfusionMatrix
    
def plot_confusion_matrix(ConfusionMatrix, name):

        mpl.style.use('seaborn')
        plt.figure(figsize= (10, 10))
        sns.heatmap(ConfusionMatrix, square=True,annot=True, cmap="BrBG", fmt="d", linewidths=.5)
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.title('{} Confusion Matrix with Labels'.format(name))
        plt.rcParams['font.size'] = '10'
        plt.grid(None)
        plt.show();

# Convolutional Neural Network

In [ ]:
name = 'Convolutional Neural Network'

In [ ]:
model = Models(Input_Shape, Epochs)
model0 = model.convolutional_neural_network()

In [ ]:
model_result_visualization(model0)

In [ ]:
TestLoss0, TestAccuracy0 = model_evulation(model0, Testing_Set)

In [ ]:
TestLoss0

In [ ]:
TestAccuracy0

In [ ]:
Real0, Prediction0 = generate_y(model0)

In [ ]:
ConfusionMatrix0 = confusion_matrix(Real0, Prediction0)

In [ ]:
ConfusionMatrix0

In [ ]:
plot_confusion_matrix(ConfusionMatrix0, name)

# Transfer Learning

**1. EfficientNetV2B0**

In [ ]:
name = 'EfficientNetV2B0'

In [ ]:
base_model = tf.keras.applications.EfficientNetV2B0(
                                                    include_top=False,
                                                    weights='imagenet')

NetV2 = Models(Input_Shape, Epochs)
model1 = NetV2.tranfer_learning(base_model)

In [ ]:
model_result_visualization(model1)

In [ ]:
TestLoss1, TestAccuracy1 = model_evulation(model1, Testing_Set)

In [ ]:
TestLoss1

In [ ]:
TestAccuracy1

In [ ]:
Real1, Prediction1 = generate_y(model1)

In [ ]:
ConfusionMatrix1 = confusion_matrix(Real1, Prediction1)

In [ ]:
plot_confusion_matrix(ConfusionMatrix1, name)

**2. ResNet101**

In [ ]:
name = 'Convolutional Neural Network'

In [ ]:
base_model = tf.keras.applications.ResNet101(
                                                    include_top=False,
                                                    weights='imagenet')

Res = Models(Input_Shape, Epochs)
model2 = Res.tranfer_learning(base_model)

In [ ]:
model_result_visualization(model2)

In [ ]:
TestLoss2, TestAccuracy2 = model_evulation(model2, Testing_Set)

In [ ]:
TestLoss2

In [ ]:
TestAccuracy2

In [ ]:
Real2, Prediction2 = generate_y(model2)

In [ ]:
ConfusionMatrix2 = confusion_matrix(Real2, Prediction2)

In [ ]:
plot_confusion_matrix(ConfusionMatrix2, name)

**3. InceptionV3**

In [ ]:
name = 'InceptionV3'

In [ ]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
        include_top=False,
        weights='imagenet',)

Inc = Models(Input_Shape, Epochs)
model3 = Inc.tranfer_learning(base_model)

In [ ]:
model_result_visualization(model3)

In [ ]:
TestLoss3, TestAccuracy3 = model_evulation(model3, Testing_Set)

In [ ]:
TestLoss3

In [ ]:
TestAccuracy3

In [ ]:
Real3, Prediction3 = generate_y(model3)

In [ ]:
ConfusionMatrix3 = confusion_matrix(Real3, Prediction3)

In [ ]:
plot_confusion_matrix(ConfusionMatrix3, name)

**4. VGG16**

In [ ]:
name = 'VGG16'

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(
        include_top=False,
        weights='imagenet',)

VGG = Models(Input_Shape, Epochs)
model4 = VGG.tranfer_learning(base_model)

In [ ]:
model_result_visualization(model4)

In [ ]:
TestLoss4, TestAccuracy4 = model_evulation(model4, Testing_Set)

In [ ]:
TestLoss4

In [ ]:
TestAccuracy4

In [ ]:
Real4, Prediction4 = generate_y(model4)

In [ ]:
ConfusionMatrix4 = confusion_matrix(Real4, Prediction4)

In [ ]:
plot_confusion_matrix(ConfusionMatrix4, name)

**5. VGG19**

In [ ]:
name = 'VGG19'

In [ ]:
base_model = tf.keras.applications.vgg19.VGG19(
        include_top=False,
        weights='imagenet',)

VGG19 = Models(Input_Shape, Epochs)
model5 = VGG19.tranfer_learning(base_model)

In [ ]:
model_result_visualization(model5)

In [ ]:
TestLoss5, TestAccuracy5 = model_evulation(model5, Testing_Set)

In [ ]:
TestLoss5

In [ ]:
TestAccuracy5

In [ ]:
Real5, Prediction5 = generate_y(model5)

In [ ]:
ConfusionMatrix5 = confusion_matrix(Real5, Prediction5)

In [ ]:
plot_confusion_matrix(ConfusionMatrix5, name)

**6. Xception**

In [ ]:
name = 'Xception'

In [ ]:
base_model = tf.keras.applications.xception.Xception(
        include_top=False,
        weights='imagenet',)

Xce = Models(Input_Shape, Epochs)
model6 = Xce.tranfer_learning(base_model)

In [ ]:
model_result_visualization(model6)

In [ ]:
TestLoss6, TestAccuracy6 = model_evulation(model6, Testing_Set)

In [ ]:
TestLoss6

In [ ]:
TestAccuracy6

In [ ]:
Real6, Prediction6 = generate_y(model6)

In [ ]:
ConfusionMatrix6 = confusion_matrix(Real6, Prediction6)

In [ ]:
plot_confusion_matrix(ConfusionMatrix6, name)

**7. EfficientNetB0**

In [ ]:
name = 'EfficientNetB0'

In [ ]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(
        include_top=False,
        weights='imagenet',)

EFF = Models(Input_Shape, Epochs)
model7 = EFF.tranfer_learning(base_model)

In [ ]:
model_result_visualization(model7)

In [ ]:
TestLoss7, TestAccuracy7 = model_evulation(model7, Testing_Set)

In [ ]:
TestLoss7

In [ ]:
TestAccuracy7

In [ ]:
Real7, Prediction7 = generate_y(model7)

In [ ]:
ConfusionMatrix7 = confusion_matrix(Real7, Prediction7)

In [ ]:
plot_confusion_matrix(ConfusionMatrix7, name)